In [2]:
import random
import sys
import datetime
from dateutil.parser import parse
from faker import Faker
from api import db, User

import pandas as pd
from api import UserHistory
from sqlalchemy import select, desc

In [ ]:
team_members_choices = ['dpo', 'IG Member', 'Work Council', 'Legal Member']
random.choice(team_members_choices)

In [3]:
user = UserHistory.query.filter_by(uid=1004983).order_by(desc(UserHistory.id))


In [4]:
result_dict = [u.__dict__ for u in user.all()]
adages_df = pd.DataFrame(result_dict)
adages_df.drop(columns=[adages_df.columns[0], 'id'], axis = 1, inplace=True)

In [5]:
adages_prev = adages_df.loc[1:]
adages_prev.reset_index(drop=True, inplace=True)
adages_next = adages_df.loc[:adages_df.shape[0]-2]
adages_diff = adages_prev.fillna('') != adages_next.fillna('')

In [6]:
adages_df

,date_of_birth,email,address,age,uid,refresh_datetime,team_member,country,phone,name,is_monitored
0,1982-11-17 21:03:55,justinbradley@example.com,"66061 Alisha Island, Stanleystad, MI 51234",72,1004983,2022-09-01 13:33:05.570334,IG Member,Palestinian Territory,6478423554,Sergio Perez,None
1,1982-11-17 21:03:55,justinbradley@example.com,"66061 Alisha Island, Stanleystad, MI 51234",72,1004983,2022-08-31 14:53:27.981232,IG Member,Palestinian Territory,6478423554,Peter Perez,None


In [7]:
adages_prev

,date_of_birth,email,address,age,uid,refresh_datetime,team_member,country,phone,name,is_monitored
0,1982-11-17 21:03:55,justinbradley@example.com,"66061 Alisha Island, Stanleystad, MI 51234",72,1004983,2022-08-31 14:53:27.981232,IG Member,Palestinian Territory,6478423554,Peter Perez,None


In [8]:
adages_next

,date_of_birth,email,address,age,uid,refresh_datetime,team_member,country,phone,name,is_monitored
0,1982-11-17 21:03:55,justinbradley@example.com,"66061 Alisha Island, Stanleystad, MI 51234",72,1004983,2022-09-01 13:33:05.570334,IG Member,Palestinian Territory,6478423554,Sergio Perez,None


In [9]:
adages_diff

,date_of_birth,email,address,age,uid,refresh_datetime,team_member,country,phone,name,is_monitored
0,False,False,False,False,False,True,False,False,False,True,False


In [10]:
adages_diff.drop(['refresh_datetime'], axis=1, inplace=True)

In [11]:
audit_trail = []
for index in adages_diff.index:
    modification = {}
    modification['MODIFIED_ON'] = adages_next.loc[index, 'refresh_datetime']
    modified_columns ={}
    for column in adages_diff.columns:
        if (adages_diff.loc[index, column]):
            modified_columns[column] = [adages_prev[column].tolist()[index], adages_next[column].tolist()[index]]
    if modified_columns : 
        modification['MODIFIED_COLUMNS']=modified_columns
        audit_trail.append(modification)

In [12]:
audit_trail

[{'MODIFIED_ON': Timestamp('2022-09-01 13:33:05.570334'),
  'MODIFIED_COLUMNS': {'name': ['Peter Perez', 'Sergio Perez']}}]

In [ ]:
adages_diff.index

In [13]:
data = {'audit_trail': 

    {'MODIFIED_ON': '14/02/2022 22:02:35',
    'ACTION': 'Update',
    'ACTION_PERFORMER':'Adages',
    'ADAGES_LIFECYCLE':[3, 1],
    'MODIFIED_COLUMNS': {
        'DATE_MONTH': ['7.0', '11.0'],
        'ADAGES_LIFECYCLE': [3, 1], 
        'NOTIFICATION': [False, True]

    }
    }

}



In [15]:
audit_trail=data['audit_trail']

In [16]:
audit_trail

{'MODIFIED_ON': '14/02/2022 22:02:35',
 'ACTION': 'Update',
 'ACTION_PERFORMER': 'Adages',
 'ADAGES_LIFECYCLE': [3, 1],
 'MODIFIED_COLUMNS': {'DATE_MONTH': ['7.0', '11.0'],
  'ADAGES_LIFECYCLE': [3, 1],
  'NOTIFICATION': [False, True]}}

In [22]:
notifications = audit_trail['MODIFIED_COLUMNS']['NOTIFICATION']
b_lifecycles = audit_trail['ADAGES_LIFECYCLE']

In [20]:
result = zip(notifications, b_lifecycles)

In [21]:
print(list(result))

[(False, 3), (True, 1)]


In [23]:
f_lifecycles = []
for notification, b_lifecycle in zip(notifications, b_lifecycles):
    f_lifecycle = int(notification*b_lifecycle) if b_lifecycle in [1, 2] else b_lifecycle
    f_lifecycles.append(f_lifecycle)


In [24]:
f_lifecycles

[3, 1]

In [17]:

audit_trail = [{'MODIFIED_ON': ('2022-09-01 11:01:15.063733'), 
  'MODIFIED_COLUMNS': {'name': ['Last Fessi', 'I am the last uid'], 'is_monitored': ['None', 'True']}},

 {'MODIFIED_ON': ('2022-08-31 15:07:45.668484'), 
  'MODIFIED_COLUMNS': {'name': ['Richard Fessi', 'Last Fessi']}}, 

 {'MODIFIED_ON': ('2022-08-31 15:00:09.444982'), 
  'MODIFIED_COLUMNS': {'name': ['Richard Taylor', 'Richard Fessi']}}]

In [18]:
for modification in audit_trail : 
    print ('this modification occured on : ', modification['MODIFIED_ON'])
    for column in modification['MODIFIED_COLUMNS']: 
        print (column+' was '+modification['MODIFIED_COLUMNS'][column][0]+' and became '+ modification['MODIFIED_COLUMNS'][column][1]) 
    

this modification occured on :  2022-09-01 11:01:15.063733
name was Last Fessi and became I am the last uid
is_monitored was None and became True
this modification occured on :  2022-08-31 15:07:45.668484
name was Richard Fessi and became Last Fessi
this modification occured on :  2022-08-31 15:00:09.444982
name was Richard Taylor and became Richard Fessi
